<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [13]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
!unzip -q concrete_data_week4.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
import os

In [16]:
classes = 2
batch_size = 100
image_size = (224,224)

In [17]:
os.listdir('concrete_data_week4')

['.DS_Store', 'test', 'train', 'valid']

In [18]:
generator = ImageDataGenerator(preprocessing_function = preprocess_input)

training_generator = generator.flow_from_directory(
    "concrete_data_week4/train",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "categorical",
)

validation_generator = generator.flow_from_directory(
    "concrete_data_week4/valid",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "categorical",
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [19]:
model = Sequential()

In [20]:
model.add(VGG16(
    include_top = False,
    pooling = 'avg',
    weights = 'imagenet',
))

In [21]:
model.add(Dense(classes, activation = 'softmax'))

In [22]:
model.layers[0].trainable = False

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

vgg16_history = model.fit_generator(
    training_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,
)

C:\Users\Diogo Reis\AppData\Local\Temp\ipykernel_10928\1719709507.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg16_history = model.fit_generator(


Epoch 1/2
301/301 [==============================] - 2036s 7s/step - loss: 0.1158 - accuracy: 0.9561 - val_loss: 0.0278 - val_accuracy: 0.9949
Epoch 2/2
301/301 [==============================] - 2024s 7s/step - loss: 0.0220 - accuracy: 0.9953 - val_loss: 0.0162 - val_accuracy: 0.9964


In [26]:
model.save("classifier_vgg16_model.h5")

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [38]:
from keras.models import load_model

model_resnet50 = load_model("classifier_resnet_model.h5", compile = False)
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
testing_generator = generator.flow_from_directory(
    "concrete_data_week4/test",
    target_size = image_size,
    shuffle = False,
)

Found 500 images belonging to 2 classes.


In [40]:
performance_vgg16 = model.evaluate_generator(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

C:\Users\Diogo Reis\AppData\Local\Temp\ipykernel_10928\1299213925.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_vgg16 = model.evaluate_generator(testing_generator)


Performance of the VGG16-trained model
Loss: 0.01366
Accuracy: 0.996


In [41]:
performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

C:\Users\Diogo Reis\AppData\Local\Temp\ipykernel_10928\2120773404.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)


Performance of the ResNet50-trained model
Loss: 0.00172
Accuracy: 1.0


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [42]:
predictions_vgg16 = model.predict_generator(testing_generator, steps = 1)

def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")

print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

C:\Users\Diogo Reis\AppData\Local\Temp\ipykernel_10928\4106598148.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_vgg16 = model.predict_generator(testing_generator, steps = 1)


First five predictions for the VGG16-trained model:
Negative (99.4% certainty)
Negative (98.8% certainty)
Negative (97.8% certainty)
Negative (98.9% certainty)
Negative (99.2% certainty)


In [43]:
predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])

C:\Users\Diogo Reis\AppData\Local\Temp\ipykernel_10928\4152097660.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)


First five predictions for the ResNet50-trained model:
Negative (100.0% certainty)
Negative (99.8% certainty)
Negative (99.8% certainty)
Negative (99.8% certainty)
Negative (100.0% certainty)


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).